In [1]:
import pandas as pd

In [59]:
reviews_wBooks_data = pd.read_parquet("c:/Users/adria/Documents/MyPortfolio/Book_Recommendation_System/Resources/DataFrames/reviews_wBooks_data.parquet")

books_data = pd.read_parquet("c:/Users/adria/Documents/MyPortfolio/Book_Recommendation_System/Resources/DataFrames/books_data.parquet")

df_ratings_books_processed = pd.read_parquet(
    "c:/Users/adria/Documents/MyPortfolio/Book_Recommendation_System/Resources/DataFrames/SentimentAnalysis/200plusRatingsPerUser_60plusRatingsPerBook/df_ratings_books_processed.parquet"
)

df_books_processed = pd.read_parquet(
    "c:/Users/adria/Documents/MyPortfolio/Book_Recommendation_System/Resources/DataFrames/SentimentAnalysis/200plusRatingsPerUser_60plusRatingsPerBook/df_books_processed.parquet"
)

In [45]:
reviews_wBooks_data.columns

Index(['title', 'review_score', 'authors', 'genre'], dtype='object')

In [46]:
books_data.columns

Index(['Title', 'description', 'authors', 'image', 'previewLink', 'publisher',
       'publishedDate', 'infoLink', 'categories', 'ratingsCount'],
      dtype='object')

In [47]:
df_ratings_books_processed.columns

Index(['Id', 'roberta_neg', 'roberta_neu', 'roberta_pos', 'book_id', 'title',
       'user_id', 'profileName', 'review_score', 'review_text',
       'roberta_compound', 'description', 'authors', 'image', 'previewLink',
       'publisher', 'infoLink', 'genre', 'publication_year'],
      dtype='object')

In [48]:
df_books_processed.columns

Index(['book_id', 'title', 'description', 'authors', 'image', 'previewLink',
       'publisher', 'infoLink', 'genre', 'publication_year'],
      dtype='object')

In [49]:

print(reviews_wBooks_data.memory_usage().sum() / (1024 * 1024), "MB")
print(books_data.memory_usage().sum() / (1024 * 1024), "MB")
print(df_ratings_books_processed.memory_usage().sum() / (1024 * 1024), "MB")
print(df_books_processed.memory_usage().sum() / (1024 * 1024), "MB")



91.5528564453125 MB
16.20526123046875 MB
1.7426223754882812 MB
0.008457183837890625 MB


# Books_data

In [60]:
books_data.rename(columns={"Title": "title", "categories": "genre"}, inplace=True)


# Review_wBooks_data

In [61]:
grouped_df = (
    reviews_wBooks_data.groupby("title")
    .agg({"title": "count", "review_score": "mean"})
    .rename(columns={"title": "title_Count", "review_score": "average_review_score"})
    .reset_index()
)

# Merge the aggregated data back to the original DataFrame
reviews_wBooks_data = pd.merge(reviews_wBooks_data, grouped_df, on="title")

# Top 10 Titles by Count and Average Score Data Frame
top_titles_10 = (
    reviews_wBooks_data.groupby("title")
    .agg({"review_score": ["count", "mean"]})
    .reset_index()
)
top_titles_10.columns = ["title", "Count", "Average_Score"]
top_titles_10 = top_titles_10.sort_values(by="Count", ascending=False).head(10)

# Prepare DataFrame for map
top_titles_12 = (
    reviews_wBooks_data.groupby("title")
    .agg({"review_score": ["count", "mean"]})
    .reset_index()
)
top_titles_12.columns = ["title", "Count", "Average_Score"]
top_titles_12 = top_titles_12.sort_values(by="Count", ascending=False).head(12)
top_titles_12.reset_index(inplace=True)

merged_df = pd.merge(
    top_titles_12,
    books_data[["title", "authors", "image", "publishedDate"]],
    on="title",
    how="inner",
)
# merged_df = merged_df.dropna()
merged_df.reset_index(inplace=True)

authors_df = pd.read_parquet("c:/Users/adria/Documents/MyPortfolio/Book_Recommendation_System/Resources/DataFrames/GenericDF/authors_df.parquet")

# Merge with locations with books df
merged_df_withloc = pd.merge(
    merged_df,
    authors_df[["authors", "lanlon", "place_of_birth", "country"]],
    on="authors",
    how="inner",
)

# Separate latitude and longitude
merged_df_withloc[["latitude", "longitude"]] = (
    merged_df_withloc["lanlon"].str.split(", ", expand=True).astype(float) # Done

)

# // TODO: Search for a better way to filter the data or outright remove it later
reviews_wBooks_data = reviews_wBooks_data[reviews_wBooks_data["authors"] != "NoAuthor"]
reviews_wBooks_data = reviews_wBooks_data[reviews_wBooks_data["genre"] != "NoGenre"]

top_genres = reviews_wBooks_data["genre"].value_counts().nlargest(3).index
df_top_genres = reviews_wBooks_data[reviews_wBooks_data["genre"].isin(top_genres)]

# Step 2: For each top genre, filter by top 10 authors
top_authors_by_genre = (
    df_top_genres.groupby("genre")["authors"]
    .value_counts()
    .groupby("genre", group_keys=False)
    .nlargest(3)
    .index.get_level_values("authors")
)

df_final_0 = df_top_genres[df_top_genres["authors"].isin(top_authors_by_genre)]

title_mapping = {
    "The Hobbit; Or, There and Back Again": "The Hobbit",
    "The Hobbitt, or there and back again; illustrated by the author.": "The Hobbit",
    "The Hobbit or There and Back Again": "The Hobbit",
    "The Hobbit There and Back Again": "The Hobbit",
}
# df_final["grouped_title"] = (
#     df_final["title"].map(title_mapping).fillna(df_final["title"])
# )
df_final = df_final_0.copy()
df_final.loc[:, "grouped_title"] = (
    df_final["title"].map(title_mapping).fillna(df_final["title"])
)

# Perform groupby operation on "grouped_title" and calculate the mean of "review_score"
df_final_grouped = (
    df_final.groupby(["genre", "authors", "grouped_title"])
    .agg({"review_score": "mean", "grouped_title": "count"})
    .rename(
        columns={
            "grouped_title": "title_Count",
            "review_score": "average_review_score",
        }
    )
    .reset_index()
)

In [63]:
df_ratings_map = merged_df_withloc.to_parquet("c:/Users/adria/Documents/MyPortfolio/Book_Recommendation_System/Resources/DataFrames/df_ratings_map.parquet")
df_graph1 = top_titles_10.to_parquet("c:/Users/adria/Documents/MyPortfolio/Book_Recommendation_System/Resources/DataFrames/df_ratings_top10.parquet")
df_graph4 = df_final_grouped.to_parquet("c:/Users/adria/Documents/MyPortfolio/Book_Recommendation_System/Resources/DataFrames/df_final_grouped.parquet")

In [53]:
top_titles_10.memory_usage().sum() / (1024 * 1024)

0.00030517578125

In [62]:
merged_df_withloc

,level_0,index,title,Count,Average_Score,authors,image,publishedDate,lanlon,place_of_birth,country,latitude,longitude
0,0,166774,The Hobbit,22023,4.657131,['J. R. R. Tolkien'],http://books.google.com/books/content?id=LLSpn...,2012,"-29.082899, 26.159786",Bloemfontein,South Africa,-29.082899,26.159786
1,1,123882,Pride and Prejudice,20371,4.527662,['Jane Austen'],http://books.google.com/books/content?id=xVeMC...,1995-01-01,"51.229223, -1.220092","Steventon, Basingstoke",UK,51.229223,-1.220092
2,11,119511,Persuasion,5498,4.386686,['Jane Austen'],http://books.google.com/books/content?id=3zsVA...,1899,"51.229223, -1.220092","Steventon, Basingstoke",UK,51.229223,-1.220092
3,3,210378,Wuthering Heights,10780,4.053711,['Emily Bronte'],http://books.google.com/books/content?id=b3ctQ...,None,"53.790707, -1.846649","Thornton, Bradford",UK,53.790707,-1.846649
4,4,164672,The Giver,7644,4.273417,['Lois Lowry'],http://books.google.com/books/content?id=Io57o...,2014-07-01,"21.314281, -157.851258","Honolulu, Hawái",USA,21.314281,-157.851258
5,5,67539,Great Expectations,7421,4.089880,['Charles Dickens'],http://books.google.com/books/content?id=CV2ZQ...,2003-01-30,"50.813335, -1.086365","Landport, Portsmouth",UK,50.813335,-1.086365
6,6,70372,Harry Potter and The Sorcerer's Stone,6796,4.739258,['J. K. Rowling'],http://books.google.com/books/content?id=HksgD...,2014-01-09,"51.541983, -2.414024","Yate, Bristol",UK,51.541983,-2.414024
7,8,26051,Brave New World,6312,4.235266,['Aldous Huxley'],http://books.google.com/books/content?id=kKh5D...,2008-12-26,"51.193015, -0.612654",Godalming,UK,51.193015,-0.612654
8,9,102197,Mere Christianity,6053,4.469684,['C. S. Lewis'],http://books.google.com/books/content?id=OF-YS...,2009-06-02,"54.595004, -5.922083",Belfast,UK,54.595004,-5.922083
9,10,175925,The Picture of Dorian Gray,5883,4.272140,['Óscar Wilde'],http://books.google.com/books/content?id=s3xej...,2016-01-24,"53.342877, -6.249795","Westland Row, Dublin",Irland,53.342877,-6.249795
